# Lecture x: Response API vs Chat Completions

## Topics to cover:
1. Why use Response API over Chat Completions
2. Streaming Responses
3. Prompt Engineering
4. Function Calling
5. A Simple Agent

It is a never-ending battle of being up-to-date with the latest changes in AI. There will always be changes to APIs and their documentation, new tools, deprecations of previous API endpoints and tools.

Our job as AI Engineers is to make sure we are up-to-date with the relevant documentation of the tools we are using. Usually, providers will give ample notice before any major changes are introduced and old methods are depreciated.

## Why the Responses API
As mentioned in OpenAIs documentation for [Responses vs Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

- Better for agentic workflows
- Stateful API
- Model availability
- Built-in tool use (Like OpenAIs own 'file search' and 'computer use')

## What the basic Chat Completions looked like

In [7]:
import os
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY is None:
    raise Exception("API key is missing")

In [3]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4.1",
  messages=[
    {"role": "developer", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who is Robert Downey Jr.?"}
  ]
)

print(completion.choices[0].message)


ChatCompletionMessage(content='Robert Downey Jr. is an acclaimed American actor and producer, widely recognized for his versatility and charismatic screen presence. Born on April 4, 1965, in New York City, he is the son of filmmaker Robert Downey Sr. and actress Elsie Ford.\n\nDowney Jr. began acting at a young age and gained early recognition for films such as "Less Than Zero" (1987) and "Chaplin" (1992), for which he received an Academy Award nomination for Best Actor. Despite a highly publicized struggle with substance abuse in the late 1990s and early 2000s, he made a celebrated comeback and is now best known for his role as Tony Stark/Iron Man in the Marvel Cinematic Universe, appearing in numerous blockbuster films starting with "Iron Man" (2008).\n\nHis other notable work includes performances in "Sherlock Holmes" (2009) and its sequel, "Tropic Thunder" (2008), and the TV series “Ally McBeal.” Downey Jr. is widely regarded as one of Hollywood\'s most talented and successful acto

In [4]:
completion = client.chat.completions.create(
  model="gpt-4.1",
  messages=[
    {"role": "developer", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is his most popular movie?"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='I’d be happy to help! Could you tell me who you are referring to? Which person’s most popular movie would you like to know about?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


## Responses API

In [15]:

response = client.responses.create(
  model="gpt-4.1",
  input=[
      {
          "role": "user",
          "content": "Who is Robert Downey Jr.?"
      }
  ]
)

print(response)


Response(id='resp_6857d48166c0819dba870a49a186a39f0f1afbe50e6030e3', created_at=1750586497.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_6857d481c2e4819d8677de7136aa52bb0f1afbe50e6030e3', content=[ResponseOutputText(annotations=[], text="**Robert Downey Jr.** is a highly acclaimed American actor and producer, born on April 4, 1965, in New York City. He is widely recognized for his versatile acting talent, charismatic screen presence, and remarkable comeback story.\n\n### Career Highlights\n- **Early Roles:** Downey began acting in his youth, appearing in films like *Less Than Zero* (1987), *Chaplin* (1992)—for which he earned an Academy Award nomination for Best Actor—and *Weird Science* (1985).\n- **Personal Struggles:** His early career was marked by critical praise but disrupted by substance abuse issues and legal troubles in the late 1990s and early 2000s.\n- **Resurgence:

In [ ]:

response = client.responses.create(
  model="gpt-4.1",
  previous_response_id=response.id,
  input=[
      {
          "role": "user",
          "content": "What is his most popular movie?"
      }
  ]
)

print(response.output_text)


Certainly! In the context of the previous response, the “3rd movie” referenced is ***Sherlock Holmes* (2009)**.

Here’s more information about **Sherlock Holmes (2009):**

---

## Sherlock Holmes (2009)

**Director:** Guy Ritchie

**Starring:**  
- **Robert Downey Jr.** as Sherlock Holmes  
- **Jude Law** as Dr. John Watson  
- **Rachel McAdams** as Irene Adler  
- **Mark Strong** as Lord Blackwood

**Genre:** Mystery, Action, Adventure

**Release Date:** December 25, 2009

---

### Plot Summary

Set in Victorian-era London, the film follows **Sherlock Holmes** (Robert Downey Jr.), a brilliant but eccentric detective, and his loyal friend and colleague **Dr. Watson** (Jude Law). Together, they investigate a series of murders linked to the mysterious occultist **Lord Blackwood** (Mark Strong), who appears to possess supernatural powers that threaten all of England. Holmes must use his trademark wit, deduction, and unorthodox methods to unravel the mystery and stop Blackwood’s deadly plo

# Prompt Engineering

In [18]:
from typing import Optional

def get_response(prompt: str, response_id: Optional[int] = None) -> tuple[str, int]:
    response = client.responses.create(
    model="gpt-4.1",
    previous_response_id=response_id,
    input=[
        {
            "role": "user",
            "content": prompt
        }
    ]
    )

    return response.output_text, response.id

### Technique 1: Write clear and concise instructions

In [19]:
PROMPT = "I need to buy a birthday present for my sister"

response_text, response_id = get_response(PROMPT)
print(response_text)

Of course! I’d love to help. To narrow it down, could you tell me:

1. How old is your sister?
2. What are her hobbies or interests?
3. What’s your budget?
4. Do you want something practical, fun, or meaningful?
5. Is there anything she needs or has mentioned wanting lately?

Let me know any details, and I’ll give you tailored gift suggestions!


In [22]:
PROMPT = "I need to buy a birthday present for my sister. She is 12 years old and is interested in ponies. Reply only with 2 options."

response_text, response_id = get_response(PROMPT)
print(response_text)

1. Breyer Classic Series Pony Model Horse Toy  
2. "Ponies & Friends" Coloring Book with Gel Pens Set


### Technique 2: Split your task into simpler subtasks

In [23]:
PROMPT = """

Write an article about the importance of muscle for skeletal health and mobility in old age.

Follow these guidelines:
- Jot down high-level bullets on the subject
- Write a short introduction
- Write a detailed section on the importance of muscle for skeletal health
- Write a detailed section on the importance of muscle for mobility
- Write a conclusion summarizing the key points
- Use markdown formatting
- Use a friendly and engaging tone
- Use simple language that is easy to understand
- Use a maximum of 500 words

"""

In [26]:
response_text, response_id = get_response(PROMPT)
print(response_text)

## High-Level Bullets

- Muscle mass naturally declines with age.
- Strong muscles support and protect bones.
- Muscle helps maintain balance and prevents falls.
- Good muscle health is key for mobility and independence.
- Exercise and nutrition can help preserve muscle.

---

## Introduction

As we grow older, it becomes more important to take care of our bodies, especially our muscles. You might not realize it, but muscles do much more than help us move—they also play a crucial role in keeping our bones healthy and making sure we can stay active and independent. In this article, we’ll explore why strong muscles are so important for skeletal health and mobility in old age.

---

## The Importance of Muscle for Skeletal Health

Muscle and bone are like best friends—they work together every day. When we use our muscles, we place good stress on our bones, which helps them stay strong. This happens because muscle contractions send signals to the bones, telling them to build more bone tiss

### Technique 3: Allow GPT to think

In [ ]:
PROMPT = """

You will be tasked to build a simple application using Python and Flask.

You MUST plan effectively, thinking through each step before you write code.

I want you to build an app that, upon clicking a button, will generate random Yoda quotes.

# Workflow

## High-level problem solving strategy
1. Deeply understand the user's requirements.
2. Break down the requirements into smaller tasks.
3. For each task, think through the implementation details.
4. Write code for each task, ensuring it works correctly.
5. Test the code to ensure it meets the requirements.

## Code implementation strategy
1. Start with the main application structure.
2. Implement the core functionality.
3. Add any additional features as needed.

## Other notes:
1. The app should look sleep and modern
2. It should feel snappy
3. Do not ask any questions
4. Give me the entire code in one file

"""

In [33]:
response_text, response_id = get_response(PROMPT)
print(response_text)

Certainly! Here’s a **well-planned solution** for your Flask app that generates Yoda quotes at the press of a button.  
It is presented in one single Python file, using Flask and modern frontend practices for a snappy UI.

Below the code, you may find **inline comments explaining crucial parts**.  
**No questions asked, no extra output.**

---

```python
from flask import Flask, render_template_string, jsonify
import random

app = Flask(__name__)

YODA_QUOTES = [
    "Do or do not. There is no try.",
    "The greatest teacher, failure is.",
    "Train yourself to let go of everything you fear to lose.",
    "Fear is the path to the dark side.",
    "Named must be your fear before banish it you can.",
    "You must unlearn what you have learned.",
    "Pass on what you have learned.",
    "Difficult to see. Always in motion is the future.",
    "Adventure. Excitement. A Jedi craves not these things.",
    "Size matters not. Look at me. Judge me by my size, do you?",
    "Much to learn, 

Let's compare with bad prompt engineering

In [ ]:
PROMPT = "I want you to build an app that, upon clicking a button, will generate random Yoda quotes. The app should be sleek and modern and snappy. Give me all the code in one python file."

response_text, response_id = get_response(PROMPT)
print(response_text)

Absolutely! Below is a complete, modern, and snappy Yoda Quote Generator app in a **single Python file** using [Streamlit](https://streamlit.io/). Streamlit allows quick, modern, interactive web apps with minimal boilerplate.

### Features
- **Modern UI:** Uses st.button, and custom st.markdown for styling.
- **Snappy Response:** Instant quote when button is clicked.
- **Randomized Yoda Quotes:** Includes a list, easy to expand.
- **Sleek look:** Simple yet elegant design.

---

#### **Instructions**
1. Save the code as `yoda_app.py`.
2. Install Streamlit:
    ```bash
    pip install streamlit
    ```
3. Run the app:
    ```bash
    streamlit run yoda_app.py
    ```

---

### **yoda_app.py**
```python
import streamlit as st
import random

# --- List of Yoda Quotes ---
YODA_QUOTES = [
    "Do or do not. There is no try.",
    "The greatest teacher, failure is.",
    "You must unlearn what you have learned.",
    "Train yourself to let go of everything you fear to lose.",
    "Fear is th